<a href="https://colab.research.google.com/github/akshayugalmogale/3d-Printing-Regression-Model/blob/main/Predict_Tension_Strength_of_3D_Print_Material.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
# Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [109]:
printer = pd.read_csv('/content/3DPrinting_data.csv')

In [110]:
printer['infill_pattern'].replace(['grid','honeycomb'], [0,1], inplace = True)
printer['material'].replace(['abs','pla'], [0,1], inplace = True)

In [111]:
#importing statsmodels library
import statsmodels.api as sm
# let's define a function for the multiple regression

def linear_Regression(x,y):
    
    x = sm.add_constant(x)
    
    #defining the model, fitting the model and printing the results
    multiple_model = sm.OLS(y,x).fit()
    print(multiple_model.summary())

# Model 2: Predicting Tension strength based on features

In [112]:
X = printer.drop(['roughness','tension_strenght','elongation'], axis = 1)
X.head()

,layer_height,wall_thickness,infill_density,infill_pattern,nozzle_temperature,bed_temperature,print_speed,material,fan_speed
0,0.02,8,90,0,220,60,40,0,0
1,0.02,7,90,1,225,65,40,0,25
2,0.02,1,80,0,230,70,40,0,50
3,0.02,4,70,1,240,75,40,0,75
4,0.02,6,90,0,250,80,40,0,100


In [113]:
y = printer['tension_strenght']
y

0     18
1     16
2      8
3     10
4      5
5     24
6     12
7     14
8     27
9     25
10    37
11    12
12    16
13     9
14    10
15    27
16    23
17    26
18    33
19    29
20    16
21    12
22    10
23    19
24     8
25    11
26    12
27    18
28    34
29    14
30    27
31    19
32    18
33     9
34    13
35    33
36    24
37    26
38    22
39     4
40    35
41    34
42    28
43    28
44    21
45    28
46    14
47    30
48    29
49    27
Name: tension_strenght, dtype: int64

In [114]:
linear_Regression(X,y)

                            OLS Regression Results                            
Dep. Variable:       tension_strenght   R-squared:                       0.673
Model:                            OLS   Adj. R-squared:                  0.609
Method:                 Least Squares   F-statistic:                     10.55
Date:                Mon, 14 Nov 2022   Prob (F-statistic):           6.91e-08
Time:                        16:26:03   Log-Likelihood:                -151.94
No. Observations:                  50   AIC:                             321.9
Df Residuals:                      41   BIC:                             339.1
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0663      0

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [115]:
#get the interaction terms by multiplying values

inter_mn = X['material']*X['nozzle_temperature']
inter_bn = X['bed_temperature']*X['nozzle_temperature']
inter_fn = X['fan_speed']*X['nozzle_temperature']
inter_fb = X['fan_speed']*X['bed_temperature']

In [116]:
#adding these interaction terms to dataset using .concat() function of pandas
#we will call this dataset as interaction

interaction = pd.concat([X,inter_mn,inter_bn,inter_fn,inter_fb], axis = 1)

#chenge column names of this interaction terms
interaction = interaction.rename(columns = {0:'interct_mn', 1:'interact_bn', 2:'interact_fn',
                             3:'interact_fb'})

interaction.head(10)

,layer_height,wall_thickness,infill_density,infill_pattern,nozzle_temperature,bed_temperature,print_speed,material,fan_speed,interct_mn,interact_bn,interact_fn,interact_fb
0,0.02,8,90,0,220,60,40,0,0,0,13200,0,0
1,0.02,7,90,1,225,65,40,0,25,0,14625,5625,1625
2,0.02,1,80,0,230,70,40,0,50,0,16100,11500,3500
3,0.02,4,70,1,240,75,40,0,75,0,18000,18000,5625
4,0.02,6,90,0,250,80,40,0,100,0,20000,25000,8000
5,0.02,10,40,1,200,60,40,1,0,200,12000,0,0
6,0.02,5,10,0,205,65,40,1,25,205,13325,5125,1625
7,0.02,10,10,1,210,70,40,1,50,210,14700,10500,3500
8,0.02,9,70,0,215,75,40,1,75,215,16125,16125,5625
9,0.02,8,40,1,220,80,40,1,100,220,17600,22000,8000


In [117]:
X = X.drop(['infill_pattern','print_speed','material'], axis = 1)
X.head()

,layer_height,wall_thickness,infill_density,nozzle_temperature,bed_temperature,fan_speed
0,0.02,8,90,220,60,0
1,0.02,7,90,225,65,25
2,0.02,1,80,230,70,50
3,0.02,4,70,240,75,75
4,0.02,6,90,250,80,100


In [118]:
linear_Regression(X,y)

                            OLS Regression Results                            
Dep. Variable:       tension_strenght   R-squared:                       0.634
Model:                            OLS   Adj. R-squared:                  0.593
Method:                 Least Squares   F-statistic:                     15.27
Date:                Mon, 14 Nov 2022   Prob (F-statistic):           1.07e-08
Time:                        16:26:04   Log-Likelihood:                -154.73
No. Observations:                  50   AIC:                             321.5
Df Residuals:                      44   BIC:                             332.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0191      0

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [119]:
interaction = pd.concat([X,inter_mn,inter_bn,inter_fn,inter_fb], axis = 1)

interaction = interaction.rename(columns = {0:'interct_mn', 1:'interact_bn', 2:'interact_fn',
                             3:'interact_fb'})

interaction.head()

,layer_height,wall_thickness,infill_density,nozzle_temperature,bed_temperature,fan_speed,interct_mn,interact_bn,interact_fn,interact_fb
0,0.02,8,90,220,60,0,0,13200,0,0
1,0.02,7,90,225,65,25,0,14625,5625,1625
2,0.02,1,80,230,70,50,0,16100,11500,3500
3,0.02,4,70,240,75,75,0,18000,18000,5625
4,0.02,6,90,250,80,100,0,20000,25000,8000


In [120]:
linear_Regression(interaction,y);

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:       tension_strenght   R-squared:                       0.718
Model:                            OLS   Adj. R-squared:                  0.663
Method:                 Least Squares   F-statistic:                     13.03
Date:                Mon, 14 Nov 2022   Prob (F-statistic):           4.06e-09
Time:                        16:26:04   Log-Likelihood:                -148.27
No. Observations:                  50   AIC:                             314.5
Df Residuals:                      41   BIC:                             331.7
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -0.0870      0

## No further improvement for tension strength in linear regression